In [ ]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# from torch.utils.data impor, DataLoader
# from torch.optim import AdamW  
# from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

# import torch
# from transformers import AutoTokenizer, GemmaForCausalLM  # נטען ישירות את Gemma, לא דרך AutoModel
# from peft import PeftModel

# # ========== CONFIG ==========
# BASE_MODEL = "google/gemma-2b"
# SAVE_DIR = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft"
# TRAIN_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_train.csv"
# VAL_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"
# TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"
# MAX_LEN = 1024
# BATCH_SIZE = 1
# EPOCHS = 3
# LR = 2e-5
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ADAPTER_PATH = "/home/liorkob/M.Sc/thesis/gemma_output/clm_lora_gpt_facts/checkpoint-12000"

# # ========== Load Model & Tokenizer ==========
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token
# tokenizer.tokenize(" yes")

# model = GemmaForCausalLM.from_pretrained(
#     BASE_MODEL,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# # ========== Load Data ==========
# df_train = pd.read_csv(TRAIN_CSV)
# df_val = pd.read_csv(VAL_CSV)
# df_test = pd.read_csv(TEST_CSV)
# print(df_train.columns)
# optimizer = AdamW(model.parameters(), lr=1e-5)  # יותר יציב


###  complete training

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, GemmaForCausalLM  # נטען ישירות את Gemma, לא דרך AutoModel
from peft import PeftModel

# ========== CONFIG ==========
BASE_MODEL = "google/gemma-3-12b-it"
SAVE_DIR = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft"
TRAIN_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_train.csv"
VAL_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"
TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"
MAX_LEN = 1024
BATCH_SIZE = 3
EPOCHS = 5
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ADAPTER_PATH = "/home/liorkob/M.Sc/thesis/gemma_output/clm_lora_gpt_facts/checkpoint-12000"

# ========== Load Data ==========
df_train = pd.read_csv(TRAIN_CSV)
df_val = pd.read_csv(VAL_CSV)
df_test = pd.read_csv(TEST_CSV)

# Memory-efficient training setup
import torch
from transformers import AutoTokenizer, GemmaForCausalLM
import gc
class DebugTrainCitationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):  # Reduced max_len
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        print(f"Building dataset with max_len={max_len}")
        print(f"Pad token ID: {self.tokenizer.pad_token_id}")
        print(f"BOS token ID: {self.tokenizer.bos_token_id}")
        print(f"EOS token ID: {self.tokenizer.eos_token_id}")
class DebugTrainCitationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):  # Reduced max_len
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            
        print(f"Building dataset with max_len={max_len}")
        print(f"Pad token ID: {self.tokenizer.pad_token_id}")
        print(f"BOS token ID: {self.tokenizer.bos_token_id}")
        print(f"EOS token ID: {self.tokenizer.eos_token_id}")

    def _clean_text(self, text):
        """Clean and preprocess text data"""
        if pd.isna(text) or text is None:
            return ""
        
        # Convert to string if not already
        text = str(text)
        
        # Basic cleaning
        text = text.strip()
        
        # Remove excessive whitespace
        text = ' '.join(text.split())
        
        # Handle Hebrew text encoding issues if needed
        try:
            # Ensure proper encoding
            text = text.encode('utf-8').decode('utf-8')
        except:
            # If encoding fails, keep original
            pass
            
        # Truncate if too long (reserve space for other parts)
        max_text_len = 400  # Adjust based on your needs
        if len(text) > max_text_len:
            text = text[:max_text_len] + "..."
            
        return text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            row = self.data.iloc[idx]
            
            # Get label
            label = int(row['label'])
            answer = "yes" if label == 1 else "no"
            
            # Get case content
            facts_a = self._clean_text(row['gpt_facts_a'])
            facts_b = self._clean_text(row['gpt_facts_b'])
            
            # Create a comprehensive prompt with actual case content
            prompt = f"""             Case A Facts: {facts_a}              Case B Facts: {facts_b}              Question: Should Case A cite Case B? Answer: """

            
            # Tokenize prompt and answer separately
            prompt_tokens = self.tokenizer.encode(prompt, add_special_tokens=True)
            answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
            
            # Combine and ensure it fits
            full_tokens = prompt_tokens + answer_tokens
            
            # Truncate prompt if too long
            if len(full_tokens) > self.max_len:
                max_prompt_len = self.max_len - len(answer_tokens) - 10  # Reserve space
                prompt_tokens = prompt_tokens[:max_prompt_len]
                full_tokens = prompt_tokens + answer_tokens
            
            # Pad to max length
            input_ids = full_tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(full_tokens))
            input_ids = input_ids[:self.max_len]
            
            # Create attention mask
            attention_mask = [1] * len(full_tokens) + [0] * (self.max_len - len(full_tokens))
            attention_mask = attention_mask[:self.max_len]
            
            # Create labels - only train on answer tokens
            labels = [-100] * len(input_ids)
            answer_start = len(prompt_tokens)
            answer_end = min(len(prompt_tokens) + len(answer_tokens), len(input_ids))
            
            for i in range(answer_start, answer_end):
                if i < len(input_ids) and input_ids[i] != self.tokenizer.pad_token_id:
                    labels[i] = input_ids[i]
            
            # Validation
            valid_labels = [l for l in labels if l != -100]
            if len(valid_labels) == 0:
                # Fallback: ensure at least one valid label
                if answer_start < len(input_ids):
                    labels[answer_start] = input_ids[answer_start]
                    valid_labels = [input_ids[answer_start]]
            
            # Debug first few examples
            if idx < 3:
                print(f"\n🔍 Example {idx}:")
                print(f"   Label: {label} -> '{answer}'")
                print(f"   Prompt length: {len(prompt_tokens)}")
                print(f"   Answer length: {len(answer_tokens)}")
                print(f"   Valid labels: {len(valid_labels)}")
                print(f"   Prompt preview: {prompt[:200]}...")
                print(f"   Answer tokens: {answer_tokens}")
                print(f"   Decoded answer: '{self.tokenizer.decode(valid_labels)}'")
            
            return {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "labels": torch.tensor(labels, dtype=torch.long)
            }
            
        except Exception as e:
            print(f"❌ Error processing example {idx}: {e}")
            print(f"Row data: {row}")
            
            # Return safe fallback
            safe_prompt = "Should cases cite each other? Answer: "
            safe_answer = "no"
            
            prompt_tokens = self.tokenizer.encode(safe_prompt, add_special_tokens=True)
            answer_tokens = self.tokenizer.encode(safe_answer, add_special_tokens=False)
            full_tokens = prompt_tokens + answer_tokens
            
            input_ids = full_tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(full_tokens))
            input_ids = input_ids[:self.max_len]
            
            attention_mask = [1] * len(full_tokens) + [0] * (self.max_len - len(full_tokens))
            attention_mask = attention_mask[:self.max_len]
            
            labels = [-100] * len(prompt_tokens) + answer_tokens + [-100] * (self.max_len - len(full_tokens))
            labels = labels[:self.max_len]
            
            return {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "labels": torch.tensor(labels, dtype=torch.long)
            }




def setup_memory_efficient_model():
    """Setup model with optimal memory usage"""
    
    # Clear everything first
    if 'model' in globals():
        del model
    if 'optimizer' in globals():
        del optimizer
    torch.cuda.empty_cache()
    gc.collect()
    
    print("🔄 Setting up memory-efficient model...")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load model with bfloat16 (better than float16, less memory than float32)
    model = GemmaForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.bfloat16,  # Good compromise
        device_map="auto",
        low_cpu_mem_usage=True  # Use less CPU memory during loading
    )
    
    # Enable gradient checkpointing to save memory
    model.gradient_checkpointing_enable()
    model.train()
    
    print(f"✅ Model loaded with {next(model.parameters()).dtype}")
    
    return model, tokenizer

import os
import torch
import gc
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, GemmaForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

def complete_training_with_validation():
    """Complete training loop with validation and checkpointing"""
    
    print("🔄 Setting up complete training pipeline...")
    
    # Clear everything first
    if 'model' in globals():
        del model
    if 'optimizer' in globals():
        del optimizer
    torch.cuda.empty_cache()
    gc.collect()
    
    # Load model with 4-bit quantization for maximum efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = GemmaForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    
    model.gradient_checkpointing_enable()
    model.train()
    
    print(f"✅ 4-bit quantized model loaded")
    
    # Create datasets with reasonable sizes
    train_dataset = DebugTrainCitationDataset(df_train, tokenizer, max_len=1024)
    val_dataset = DebugTrainCitationDataset(df_val, tokenizer, max_len=1024)
    
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    
    # SGD optimizer for memory efficiency
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)
    
    # Create save directory
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    print("🚀 Starting complete training...")
    
    # Training metrics
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    model.train()
    step_count = 0
    
    for epoch in range(EPOCHS):
        print(f"\n📚 Epoch {epoch + 1}/{EPOCHS}")
        
        epoch_train_loss = 0
        valid_batches = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        
        for batch in progress_bar:
            step_count += 1
            
            # Validation every 50 steps
            if step_count % 1000 == 0:
                val_loss = run_validation(model, val_loader, tokenizer)
                val_losses.append(val_loss)
                
                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    print(f"💾 New best validation loss: {val_loss:.4f}, saving model...")
                    model.save_pretrained(f"{SAVE_DIR}/best_model")
                    tokenizer.save_pretrained(f"{SAVE_DIR}/best_model")
                
                model.train()  # Return to training mode
                        
            # Training step
            try:
                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)
                labels = batch["labels"].to(model.device)
                
                # Check for valid labels
                valid_label_count = (labels != -100).sum().item()
                if valid_label_count == 0:
                    continue
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                
                if torch.isnan(loss):
                    print(f"⚠️ NaN loss at step {step_count}, skipping...")
                    continue
                
                # Backward pass
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                # Update
                optimizer.step()
                optimizer.zero_grad()
                
                # Track metrics
                epoch_train_loss += loss.item()
                valid_batches += 1
                train_losses.append(loss.item())
                
                # Progress reporting
                if step_count % 10 == 0:
                    memory_used = torch.cuda.memory_allocated() / 1024**3
                    progress_bar.set_postfix({
                        'loss': f'{loss.item():.4f}',
                        'step': step_count,
                        'memory': f'{memory_used:.2f}GB'
                    })
                
                # Clear cache periodically
                if step_count % 20 == 0:
                    torch.cuda.empty_cache()
                
                # Early stopping for testing
                # if step_count >= 500:  # Limit for initial run
                #     print(f"\n🛑 Stopping at step {step_count} for testing")
                #     break
                    
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"❌ OOM at step {step_count}, clearing cache...")
                    torch.cuda.empty_cache()
                    optimizer.zero_grad()
                    continue
                else:
                    raise e
        
        # Epoch summary
        if valid_batches > 0:
            avg_train_loss = epoch_train_loss / valid_batches
            print(f"\n📊 Epoch {epoch+1} Summary:")
            print(f"   Average train loss: {avg_train_loss:.4f}")
            print(f"   Valid batches: {valid_batches}")
            print(f"   Total steps: {step_count}")
        
        # if step_count >= 500:  # Break from epoch loop too
        #     break
    
    # Final validation
    print("\n🔍 Running final validation...")
    final_val_loss = run_validation(model, val_loader, tokenizer)
    
    # Save final model
    print("💾 Saving final model...")
    model.save_pretrained(f"{SAVE_DIR}/final_model")
    tokenizer.save_pretrained(f"{SAVE_DIR}/final_model")
    
    # Training summary
    print(f"\n🎉 Training completed!")
    print(f"   Total steps: {step_count}")
    print(f"   Best validation loss: {best_val_loss:.4f}")
    print(f"   Final validation loss: {final_val_loss:.4f}")
    print(f"   Models saved to: {SAVE_DIR}")
    
    return model, tokenizer, train_losses, val_losses

def run_validation(model, val_loader, tokenizer):
    """Run validation and return average loss"""
    model.eval()
    total_val_loss = 0
    valid_val_batches = 0
    
    print(f"\n🔍 Running validation...")
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=False):
            try:
                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)
                labels = batch["labels"].to(model.device)
                
                # Check for valid labels
                valid_label_count = (labels != -100).sum().item()
                if valid_label_count == 0:
                    continue
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                
                if not torch.isnan(loss):
                    total_val_loss += loss.item()
                    valid_val_batches += 1
                
                # Limit validation steps to save time
                if valid_val_batches >= 50:
                    break
                    
            except RuntimeError as e:
                if "out of memory" in str(e):
                    torch.cuda.empty_cache()
                    continue
                else:
                    raise e
    
    avg_val_loss = total_val_loss / max(valid_val_batches, 1)
    print(f"   Validation loss: {avg_val_loss:.4f} (from {valid_val_batches} batches)")
    
    return avg_val_loss

def plot_training_curves(train_losses, val_losses):
    """Plot training and validation curves"""
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(12, 4))
        
        # Training loss
        plt.subplot(1, 2, 1)
        plt.plot(train_losses)
        plt.title('Training Loss')
        plt.xlabel('Step')
        plt.ylabel('Loss')
        plt.grid(True)
        
        # Validation loss
        plt.subplot(1, 2, 2)
        if val_losses:
            plt.plot(range(0, len(train_losses), 50), val_losses)
            plt.title('Validation Loss')
            plt.xlabel('Step')
            plt.ylabel('Loss')
            plt.grid(True)
        
        plt.tight_layout()
        plt.savefig(f"{SAVE_DIR}/training_curves.png")
        plt.show()
        
        print(f"📈 Training curves saved to {SAVE_DIR}/training_curves.png")
        
    except ImportError:
        print("📈 Matplotlib not available, skipping plots")


# Check GPU memory first
def check_gpu_memory():
    """Check available GPU memory"""
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        free = total_memory - reserved
        
        print(f"GPU Memory Status:")
        print(f"  Total: {total_memory:.2f}GB")
        print(f"  Allocated: {allocated:.2f}GB") 
        print(f"  Reserved: {reserved:.2f}GB")
        print(f"  Free: {free:.2f}GB")
        
        return free > 2.0  # Need at least 2GB free
    return False

# Run the appropriate training based on memory
print("Checking GPU memory...")
has_enough_memory = check_gpu_memory()

print("Starting complete training pipeline...")
model, tokenizer, train_losses, val_losses = complete_training_with_validation()

# Plot results
plot_training_curves(train_losses, val_losses)



train- no lora

In [1]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_cosine_schedule_with_warmup, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tqdm import tqdm
import json

# ========== CONFIG FOR GEMMA 3 ==========
BASE_MODEL = "google/gemma-3-12b-it"  # Real Gemma 3
SAVE_DIR = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft-improved"
TRAIN_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_train.csv"
VAL_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"
TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"

# Training settings
BATCH_SIZE = 1
GRAD_ACCUMULATION = 8
EPOCHS = 3
LEARNING_RATE = 1e-4
MAX_LEN = 4096  # Reasonable compromise - 4x original without OOM

# LoRA settings
LORA_R = 32
LORA_ALPHA = 64

os.makedirs(SAVE_DIR, exist_ok=True)

# ========== DATASET ==========
class CitationDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len=1024, is_training=True):
        self.data = pd.read_csv(csv_path).dropna()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_training = is_training
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        label = int(row['label'])
        answer = "yes" if label == 1 else "no"
        
        # Smart truncation - keep more text but avoid OOM
        facts_a = str(row['gpt_facts_a'])[:1500]  # Much more than 400, but safe
        facts_b = str(row['gpt_facts_b'])[:1500]  # Much more than 400, but safe
        
        prompt = f"Case A: {facts_a}\nCase B: {facts_b}\nShould Case A cite Case B? Answer: "
        
        if self.is_training:
            full_text = prompt + answer
            tokens = self.tokenizer.encode(full_text, max_length=self.max_len, truncation=True, add_special_tokens=True)
            prompt_tokens = self.tokenizer.encode(prompt, max_length=self.max_len-10, truncation=True, add_special_tokens=True)
            
            # Ensure we have space for the answer
            if len(prompt_tokens) >= self.max_len - 5:
                prompt_tokens = prompt_tokens[:self.max_len-5]
            
            answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
            full_tokens = prompt_tokens + answer_tokens
            
            input_ids = full_tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(full_tokens))
            attention_mask = [1] * len(full_tokens) + [0] * (self.max_len - len(full_tokens))
            
            # Only train on answer tokens
            labels = [-100] * len(prompt_tokens) + answer_tokens + [-100] * (self.max_len - len(full_tokens))
            
            return {
                "input_ids": torch.tensor(input_ids[:self.max_len]),
                "attention_mask": torch.tensor(attention_mask[:self.max_len]),
                "labels": torch.tensor(labels[:self.max_len])
            }
        else:
            tokens = self.tokenizer.encode(prompt, max_length=self.max_len, truncation=True, add_special_tokens=True)
            input_ids = tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(tokens))
            attention_mask = [1] * len(tokens) + [0] * (self.max_len - len(tokens))
            
            return {
                "input_ids": torch.tensor(input_ids[:self.max_len]),
                "attention_mask": torch.tensor(attention_mask[:self.max_len]),
                "true_label": torch.tensor(label),
                "prompt_length": len(tokens)
            }

# ========== MODEL SETUP ==========
def setup_model():
    print(f"Loading {BASE_MODEL}...")
    
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Model with quantization for Gemma 3
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    
    try:
        model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            attn_implementation="eager"  # More stable for Gemma 3
        )
    except Exception as e:
        print(f"Error loading Gemma 3: {e}")
        print("This might be due to Gemma 3 compatibility issues...")
        raise e
    
    # LoRA for Gemma 3
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.1
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    return model, tokenizer

# ========== EVALUATION ==========
def evaluate(model, eval_loader, tokenizer):
    model.eval()
    predictions, true_labels = [], []
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(eval_loader, desc="Evaluating")):
            if i >= 100:  # Limit evaluation to save time
                break
                
            try:
                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)
                true_label = batch["true_label"].item()
                prompt_length = batch["prompt_length"].item()
                
                # Clear cache before generation
                torch.cuda.empty_cache()
                
                # Generate with simpler settings
                outputs = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=3,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    use_cache=False  # Disable cache to avoid consistency issues
                )
                
                # Parse response
                generated_text = tokenizer.decode(outputs[0][prompt_length:], skip_special_tokens=True).lower().strip()
                pred = 1 if "yes" in generated_text else 0
                
                predictions.append(pred)
                true_labels.append(true_label)
                
            except Exception as e:
                print(f"Evaluation error: {e}, skipping sample")
                # Default prediction
                predictions.append(0)
                true_labels.append(true_label)
                continue
    
    # Handle empty predictions
    if not predictions:
        return {'accuracy': 0.0, 'f1': 0.0, 'precision': 0.0, 'recall': 0.0}
    
    # Metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predictions),
        'f1': f1_score(true_labels, predictions, zero_division=0),
        'precision': precision_score(true_labels, predictions, zero_division=0),
        'recall': recall_score(true_labels, predictions, zero_division=0)
    }
    
    model.train()
    return metrics

# ========== TRAINING ==========
def train():
    # Setup
    model, tokenizer = setup_model()
    
    # Data
    train_dataset = CitationDataset(TRAIN_CSV, tokenizer, is_training=True)
    val_dataset = CitationDataset(VAL_CSV, tokenizer, is_training=False)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    
    # Optimizer & Scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_loader) * EPOCHS // GRAD_ACCUMULATION
    scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=total_steps // 10, num_training_steps=total_steps
    )
    
    # Training loop
    best_f1 = 0
    step = 0
    
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        
        for i, batch in enumerate(tqdm(train_loader)):
            try:
                input_ids = batch["input_ids"].to(model.device)
                attention_mask = batch["attention_mask"].to(model.device)
                labels = batch["labels"].to(model.device)
                
                # Check for NaN in inputs
                if torch.isnan(input_ids).any() or torch.isnan(attention_mask).any() or torch.isnan(labels).any():
                    print(f"NaN detected in inputs at step {step}, skipping...")
                    continue
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss / GRAD_ACCUMULATION
                
                # Check for NaN loss
                if torch.isnan(loss):
                    print(f"NaN loss at step {step}, skipping...")
                    optimizer.zero_grad()
                    continue
                
                loss.backward()
                
                if (i + 1) % GRAD_ACCUMULATION == 0:
                    # Clip gradients to prevent explosion
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    step += 1
                    
                    if step % 100 == 0:
                        print(f"Step {step}, Loss: {loss.item():.4f}")
                
                # Clear cache periodically
                if step % 50 == 0:
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                print(f"Training error at step {step}: {e}, skipping batch...")
                optimizer.zero_grad()
                torch.cuda.empty_cache()
                continue
            
            # Validation every 500 steps
            if step % 500 == 0 and step > 0:
                metrics = evaluate(model, val_loader, tokenizer)
                print(f"Validation - F1: {metrics['f1']:.4f}, Acc: {metrics['accuracy']:.4f}")
                
                if metrics['f1'] > best_f1:
                    best_f1 = metrics['f1']
                    model.save_pretrained(f"{SAVE_DIR}/best_model")
                    tokenizer.save_pretrained(f"{SAVE_DIR}/best_model")
                    print(f"New best F1: {best_f1:.4f}")
    
    # Final save
    model.save_pretrained(f"{SAVE_DIR}/final_model")
    tokenizer.save_pretrained(f"{SAVE_DIR}/final_model")
    
    # Test evaluation
    test_dataset = CitationDataset(TEST_CSV, tokenizer, is_training=False)
    test_loader = DataLoader(test_dataset, batch_size=1)
    
    final_metrics = evaluate(model, test_loader, tokenizer)
    print(f"\nFinal Test Results:")
    for k, v in final_metrics.items():
        print(f"  {k}: {v:.4f}")
    
    # Save results
    with open(f"{SAVE_DIR}/results.json", 'w') as f:
        json.dump(final_metrics, f, indent=2)

if __name__ == "__main__":
    train()

/home/liorkob/.conda/envs/4gemma_v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading google/gemma-3-12b-it...


Loading checkpoint shards: 100%|██████████| 5/5 [00:08<00:00,  1.69s/it]


trainable params: 136,912,896 || all params: 12,324,237,936 || trainable%: 1.1109

Epoch 1/3


 20%|█▉        | 799/4052 [11:32<47:12,  1.15it/s]  

...

Validation - F1: 0.9118, Acc: 0.9400


The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are

Validation - F1: 0.9118, Acc: 0.9400


Evaluating:  11%|█▏        | 100/870 [01:51<14:15,  1.11s/it]


Final Test Results:
  accuracy: 0.9300
  f1: 0.8852
  precision: 0.8438
  recall: 0.9310


In [2]:
# Gemma-3 (LoRA) — single-cell generation & metrics
import os, json, torch, pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM

# ====== CONFIG (edit if needed) ======
MODEL_DIR = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft-improved/best_model"  # or .../final_model
VAL_CSV  = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"
TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"
SPLIT = "test"      # "val" or "test"
MAX_LEN = 4096
MAX_NEW_TOKENS = 3
SHOW_N = 10         # printed samples
LIMIT = None        # e.g., 200 to subset
# =====================================

OUT_DIR = os.path.join(MODEL_DIR, "generations")
os.makedirs(OUT_DIR, exist_ok=True)

# Load model & tokenizer (4-bit like QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
print(f"Loading adapter from: {MODEL_DIR}")
model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    trust_remote_code=True,
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def build_prompt(facts_a: str, facts_b: str) -> str:
    facts_a = str(facts_a)[:1500]
    facts_b = str(facts_b)[:1500]
    return f"Case A: {facts_a}\nCase B: {facts_b}\nShould Case A cite Case B? Answer: "

# Load data
csv_path = TEST_CSV if SPLIT == "test" else VAL_CSV
df = pd.read_csv(csv_path).dropna()
if LIMIT is not None:
    df = df.iloc[:LIMIT].copy()
print(f"Loaded {len(df)} rows from {csv_path}")

# Generate
rows_out, preds, golds = [], [], []
shown = 0
model.eval()
with torch.no_grad():
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Generating[{SPLIT}]"):
        label = int(row["label"])
        prompt = build_prompt(row["gpt_facts_a"], row["gpt_facts_b"])
        enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LEN, add_special_tokens=True)
        input_ids = enc["input_ids"].to(model.device)
        attention_mask = enc["attention_mask"].to(model.device)
        prompt_len = input_ids.shape[1]

        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,
        )

        gen_text = tokenizer.decode(out[0][prompt_len:], skip_special_tokens=True).strip().lower()
        pred = 1 if "yes" in gen_text else 0

        preds.append(pred)
        golds.append(label)
        rows_out.append({
            "idx": idx,
            "prompt": prompt,
            "generated_text": gen_text,
            "pred": pred,
            "true_label": label,
        })

        if shown < SHOW_N:
            print("\n--- SAMPLE ---")
            print(prompt)
            print(f"Model → {gen_text!r} | pred={pred} | gold={label}")
            shown += 1

# Metrics
metrics = {
    "accuracy": accuracy_score(golds, preds),
    "f1": f1_score(golds, preds, zero_division=0),
    "precision": precision_score(golds, preds, zero_division=0),
    "recall": recall_score(golds, preds, zero_division=0),
    "n_samples": len(golds),
}

# Save
out_csv = os.path.join(OUT_DIR, f"gen_{SPLIT}.csv")
pd.DataFrame(rows_out).to_csv(out_csv, index=False)
with open(out_csv.replace(".csv", "_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

print("\n== Metrics ==")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")
print(f"\nSaved: {out_csv}")
print(f"Saved: {out_csv.replace('.csv', '_metrics.json')}")

# Show a quick peek as the cell output
pd.DataFrame(rows_out).head()


Loading adapter from: /home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft-improved/best_model


Loading checkpoint shards: 100%|██████████| 5/5 [02:50<00:00, 34.13s/it]


Loaded 870 rows from /home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv


Generating[test]:   0%|          | 1/870 [00:02<29:55,  2.07s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- SAMPLE ---
Case A: הנאשם הורשע על פי הודאתו בעבירה שעניינה החזקת סם לשימוש עצמי, לפי סעיף 7(א)+7(ג) סיפא לפקודת הסמים המסוכנים (נוסח חדש), תשל"ג- 1973 (להלן – פקודת הסמים). בהתאם לעובדות כתב האישום המתוקן, ביום 29.6.21 החזיק הנאשם בביתו ולשימושו האישי סמים כמפורט להלן: חשיש במשקל 53.93 גרם; קנבוס במשקל 7.75 גרם; קוקאין במשקל 0.0808 גרם; MDMA – קטאמין במשקל  0.3615 גרם.  
להשלמת התמונה יצוין כי הודאת הנאשם באה אגב הסדר טיעון במסגרתו כתב האישום תוקן לעבירת שימוש עצמי והנאשם הופנה לשירות המבחן. הצדדים הסכימו כי במקרה של תסקיר חיובי, המאסר המותנה התלוי ועומד נגד הנאשם יוארך ואילו במקרה של תסקיר שלילי, תטען המאשימה לעונש בעבודות שירות וההגנה תטען באופן חופשי. אציין כבר עתה כי בסופו של יום וויתר הנאשם על קבלת תסקיר והצדדים טענו לעונש על בסיס החלופה השנייה של ההסדר. כן אציין שלאחר שלב הטיעונים לעונש הפניתי את הנאשם לממונה על עבודות השירות אך הוא לא התייצב לראיון שלוש פעמים. בשלב זה הוחלף הייצוג והתבקשה הפניה חוזרת לשירות המבחן אותה דחיתי לאור התנהגות הנאשם כמפורט לעיל. עם זאת נעתרתי לבקש

...

Generating[test]:   1%|          | 10/870 [00:12<16:51,  1.18s/it]The following generation flags are not valid and may be ignored: ['top_p', 'top_k', 'cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- SAMPLE ---
Case A: הנאשם הודה והורשע, במסגרת הסדר טיעון, בעובדות כתב אישום מתוקן בעבירות של החזקה/שימוש בסמים שלא לצריכה עצמית - עבירה לפי סעיף 7 (א)+(ג) לפקודת הסמים המסוכנים (נוסח חדש), תשל"ג 1973 ועבירה של החזקת כלים להכנת סם - עבירה לפי סעיף 10 לצו בדבר הסמים המסוכנים, תשל"ה 1975, החזקת סמים לשימוש עצמי (3 פרטי אישום), עבירה לפי סעיף 7 (א)+(ג) לפקודת הסמים המסוכנים, עבירה של קשירת קשר לעשות פשע - עבירה לפי סעיף 499(א)(1) לחוק העונשין, התשל"ז 1977.  
הצדדים הגיעו להסכמה עונשית בתיק שלפיה ייגזרו על הנאשם 9 חודשי מאסר בפועל בניכוי ימי מעצרו, קנס כספי בסך 50,000 ₪, מאסר על תנאי בגין ביצוע עבירות הסמים וכן מאסר על תנאי, בן חודש ימים, בגין ביצוע עבירת איומים.
Case B: ביום 29.06.2022 הורשע הנאשם על פי הודאתו בעובדות כתב האישום, בביצוע עבירת שוד בנסיבות מחמירות - עבירה לפי סעיף 402(ב) לחוק העונשין, התשל"ז-1977 (להלן: "החוק"). הודאת הנאשם ניתנה במסגרת הסדר טיעון אשר לא כלל הסכמה עונשית. נקבע כי לעניין זה יטען כל צד את טענותיו באופן חופשי.

בתאריך 14.01.2022 בשעה 17:15 לערך סמוך למגדלור

Generating[test]: 100%|██████████| 870/870 [16:05<00:00,  1.11s/it]


== Metrics ==
accuracy: 0.9241
f1: 0.8874
precision: 0.8814
recall: 0.8935
n_samples: 870

Saved: /home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft-improved/best_model/generations/gen_test.csv
Saved: /home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft-improved/best_model/generations/gen_test_metrics.json


,idx,prompt,generated_text,pred,true_label
0,0,Case A: הנאשם הורשע על פי הודאתו בעבירה שעניינ...,yesyesyes,1,1
1,1,Case A: הנאשם הורשע על יסוד הודאתו בעבירות של ...,nonono,0,0
2,2,Case A: הנאשם יליד 1987 הורשע על פי הודאתו במס...,nonono,0,0
3,3,"Case A: על-פי הודאתו, הנאשם הורשע בעבירה של הח...",nonono,0,0
4,4,Case A: הנאשם הורשע על פי הודאתו אשר ניתנה במס...,nonono,0,0


### eval

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from transformers import AutoTokenizer, GemmaForCausalLM
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
BASE_MODEL = "google/gemma-2b"
MODEL_PATH = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft/final_model"  # or best_model
TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"
VAL_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EvaluationDataset(Dataset):
    """Dataset for evaluation - simpler than training dataset"""
    def __init__(self, dataframe, tokenizer, max_len=1024):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
    def _clean_text(self, text):
        """Clean and preprocess text data"""
        if pd.isna(text) or text is None:
            return ""
        
        # Convert to string if not already
        text = str(text)
        
        # Basic cleaning
        text = text.strip()
        
        # Remove excessive whitespace
        text = ' '.join(text.split())
        
        # Handle Hebrew text encoding issues if needed
        try:
            # Ensure proper encoding
            text = text.encode('utf-8').decode('utf-8')
        except:
            # If encoding fails, keep original
            pass
            
        # Truncate if too long (reserve space for other parts)
        max_text_len = 400  # Adjust based on your needs
        if len(text) > max_text_len:
            text = text[:max_text_len] + "..."
            
        return text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
                
    # Create prompt (same as training)
        row = self.data.iloc[idx]
        
        # Get label
        label = int(row['label'])
        true_label = "yes" if label == 1 else "no"
        true_label = 1 if label == 1 else 0

        # Get case content
        facts_a = self._clean_text(row['gpt_facts_a'])
        facts_b = self._clean_text(row['gpt_facts_b'])
        
        # Create a comprehensive prompt with actual case content
        prompt = (
            f"Case A Facts: {facts_a}\n"
            f"Case B Facts: {facts_b}\n"
            f"Question: Should Case A cite Case B?\n"
            f"Answer: yes or no?\n"
        )
        # Tokenize prompt only (we'll generate the answer)
        prompt_tokens = self.tokenizer.encode(prompt, add_special_tokens=True)
        
        # Pad to max length
        input_ids = prompt_tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(prompt_tokens))
        input_ids = input_ids[:self.max_len]
        
        attention_mask = [1] * len(prompt_tokens) + [0] * (self.max_len - len(prompt_tokens))
        attention_mask = attention_mask[:self.max_len]
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "true_label": torch.tensor(true_label, dtype=torch.long),
            "prompt_length": len(prompt_tokens)
        }
def load_trained_model(model_path):
    """Load the trained model and tokenizer"""
    print(f"🔄 Loading model from {model_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = GemmaForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    
    model.eval()
    print("✅ Model loaded successfully")
    return model, tokenizer

def predict_single_example(model, tokenizer, input_ids, attention_mask, prompt_length):
    """Predict for a single example"""
    with torch.no_grad():
        # Generate response
        generated = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10,  # We only expect "yes" or "no"
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.1,
            top_p=0.9
        )
        
        # Extract only the generated part (after prompt)
        generated_text = tokenizer.decode(generated[0][prompt_length:], skip_special_tokens=True).strip().lower()
        
        # Parse prediction
        if "yes" in generated_text:
            prediction = 1
            confidence = 0.8  # We could calculate actual probabilities if needed
        elif "no" in generated_text:
            prediction = 0
            confidence = 0.8
        else:
            # Default to "no" if unclear
            prediction = 0
            confidence = 0.5
            
        return prediction, confidence, generated_text

def evaluate_model(model, tokenizer, dataloader, dataset_name="Test"):
    """Comprehensive model evaluation"""
    print(f"\n🔍 Evaluating on {dataset_name} set...")
    
    predictions = []
    true_labels = []
    confidences = []
    generated_texts = []
    
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating {dataset_name}"):
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            true_label = batch["true_label"].item()
            prompt_length = batch["prompt_length"].item()
            
            try:
                pred, conf, gen_text = predict_single_example(
                    model, tokenizer, input_ids, attention_mask, prompt_length
                )
                
                predictions.append(pred)
                true_labels.append(true_label)
                confidences.append(conf)
                generated_texts.append(gen_text)
                
            except Exception as e:
                print(f"Error processing example: {e}")
                # Default prediction
                predictions.append(0)
                true_labels.append(true_label)
                confidences.append(0.5)
                generated_texts.append("error")
    
    return np.array(predictions), np.array(true_labels), np.array(confidences), generated_texts

def calculate_metrics(predictions, true_labels, confidences):
    """Calculate comprehensive metrics"""
    metrics = {}
    
    # Basic metrics
    metrics['accuracy'] = accuracy_score(true_labels, predictions)
    metrics['precision'] = precision_score(true_labels, predictions, zero_division=0)
    metrics['recall'] = recall_score(true_labels, predictions, zero_division=0)
    metrics['f1'] = f1_score(true_labels, predictions, zero_division=0)
    
    # AUC (using confidence scores as probabilities)
    try:
        # Convert predictions and confidences to probability scores
        prob_scores = np.where(predictions == 1, confidences, 1 - confidences)
        metrics['auc'] = roc_auc_score(true_labels, prob_scores)
    except:
        metrics['auc'] = 0.5
    
    # Class-wise metrics
    report = classification_report(true_labels, predictions, output_dict=True, zero_division=0)
    metrics['class_0_precision'] = report['0']['precision']
    metrics['class_0_recall'] = report['0']['recall']
    metrics['class_0_f1'] = report['0']['f1-score']
    metrics['class_1_precision'] = report['1']['precision']
    metrics['class_1_recall'] = report['1']['recall']
    metrics['class_1_f1'] = report['1']['f1-score']
    
    # Confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    metrics['confusion_matrix'] = cm
    
    return metrics

def plot_results(val_metrics, test_metrics, save_path):
    """Plot evaluation results"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Metrics comparison
    metric_names = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    val_scores = [val_metrics[m] for m in metric_names]
    test_scores = [test_metrics[m] for m in metric_names]
    
    x = np.arange(len(metric_names))
    width = 0.35
    
    axes[0, 0].bar(x - width/2, val_scores, width, label='Validation', alpha=0.8)
    axes[0, 0].bar(x + width/2, test_scores, width, label='Test', alpha=0.8)
    axes[0, 0].set_xlabel('Metrics')
    axes[0, 0].set_ylabel('Score')
    axes[0, 0].set_title('Model Performance Comparison')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(metric_names)
    axes[0, 0].legend()
    axes[0, 0].set_ylim(0, 1)
    
    # Add value labels on bars
    for i, (val_score, test_score) in enumerate(zip(val_scores, test_scores)):
        axes[0, 0].text(i - width/2, val_score + 0.01, f'{val_score:.3f}', 
                       ha='center', va='bottom', fontsize=8)
        axes[0, 0].text(i + width/2, test_score + 0.01, f'{test_score:.3f}', 
                       ha='center', va='bottom', fontsize=8)
    
    # 2. Validation Confusion Matrix
    sns.heatmap(val_metrics['confusion_matrix'], annot=True, fmt='d', 
                cmap='Blues', ax=axes[0, 1])
    axes[0, 1].set_title('Validation Confusion Matrix')
    axes[0, 1].set_xlabel('Predicted')
    axes[0, 1].set_ylabel('True')
    
    # 3. Test Confusion Matrix
    sns.heatmap(test_metrics['confusion_matrix'], annot=True, fmt='d', 
                cmap='Blues', ax=axes[1, 0])
    axes[1, 0].set_title('Test Confusion Matrix')
    axes[1, 0].set_xlabel('Predicted')
    axes[1, 0].set_ylabel('True')
    
    # 4. Class-wise F1 scores
    class_metrics = ['class_0_f1', 'class_1_f1']
    val_class_scores = [val_metrics[m] for m in class_metrics]
    test_class_scores = [test_metrics[m] for m in class_metrics]
    
    x = np.arange(len(class_metrics))
    axes[1, 1].bar(x - width/2, val_class_scores, width, label='Validation', alpha=0.8)
    axes[1, 1].bar(x + width/2, test_class_scores, width, label='Test', alpha=0.8)
    axes[1, 1].set_xlabel('Class')
    axes[1, 1].set_ylabel('F1 Score')
    axes[1, 1].set_title('Class-wise F1 Scores')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(['Class 0 (No)', 'Class 1 (Yes)'])
    axes[1, 1].legend()
    axes[1, 1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

def print_detailed_results(metrics, dataset_name):
    """Print detailed evaluation results"""
    print(f"\n📊 {dataset_name} Results:")
    print("=" * 50)
    print(f"Accuracy:     {metrics['accuracy']:.4f}")
    print(f"Precision:    {metrics['precision']:.4f}")
    print(f"Recall:       {metrics['recall']:.4f}")
    print(f"F1 Score:     {metrics['f1']:.4f}")
    print(f"AUC:          {metrics['auc']:.4f}")
    print()
    print("Class-wise metrics:")
    print(f"Class 0 (No):  Precision={metrics['class_0_precision']:.4f}, "
          f"Recall={metrics['class_0_recall']:.4f}, F1={metrics['class_0_f1']:.4f}")
    print(f"Class 1 (Yes): Precision={metrics['class_1_precision']:.4f}, "
          f"Recall={metrics['class_1_recall']:.4f}, F1={metrics['class_1_f1']:.4f}")
    print()
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])

def main():
    """Main evaluation pipeline"""
    
    # Load data
    print("📁 Loading data...")
    df_val = pd.read_csv(VAL_CSV)
    df_test = pd.read_csv(TEST_CSV)
    
    print(f"Validation set size: {len(df_val)}")
    print(f"Test set size: {len(df_test)}")
    
    # Load model
    model, tokenizer = load_trained_model(MODEL_PATH)
    
    # Create datasets
    val_dataset = EvaluationDataset(df_val, tokenizer)
    test_dataset = EvaluationDataset(df_test, tokenizer)
    
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Evaluate on validation set
    val_preds, val_true, val_conf, val_texts = evaluate_model(model, tokenizer, val_loader, "Validation")
    val_metrics = calculate_metrics(val_preds, val_true, val_conf)
    
    # Evaluate on test set
    test_preds, test_true, test_conf, test_texts = evaluate_model(model, tokenizer, test_loader, "Test")
    test_metrics = calculate_metrics(test_preds, test_true, test_conf)
    
    # Print results
    print_detailed_results(val_metrics, "Validation")
    print_detailed_results(test_metrics, "Test")
    
    # Plot results
    save_dir = os.path.dirname(MODEL_PATH)
    plot_results(val_metrics, test_metrics, f"{save_dir}/evaluation_results.png")
    
    # Save detailed results
    results_summary = {
        'validation_metrics': val_metrics,
        'test_metrics': test_metrics,
        'sample_predictions': {
            'val_examples': list(zip(val_texts[:10], val_preds[:10], val_true[:10])),
            'test_examples': list(zip(test_texts[:10], test_preds[:10], test_true[:10]))
        }
    }
    
    # Save as CSV for easy analysis
    results_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC'],
        'Validation': [val_metrics['accuracy'], val_metrics['precision'], 
                      val_metrics['recall'], val_metrics['f1'], val_metrics['auc']],
        'Test': [test_metrics['accuracy'], test_metrics['precision'], 
                test_metrics['recall'], test_metrics['f1'], test_metrics['auc']]
    })
    
    results_df.to_csv(f"{save_dir}/evaluation_metrics.csv", index=False)
    
    print(f"\n💾 Results saved to:")
    print(f"   - Plots: {save_dir}/evaluation_results.png")
    print(f"   - Metrics: {save_dir}/evaluation_metrics.csv")
    
    # Sample predictions analysis
    print(f"\n🔍 Sample Predictions (Test Set):")
    print("-" * 60)
    for i in range(min(5, len(test_texts))):
        gen_text = test_texts[i]
        pred = test_preds[i]
        true = test_true[i]
        status = "✅" if pred == true else "❌"
        print(f"{i+1}. Generated: '{gen_text}' | Predicted: {pred} | True: {true} {status}")
    
    return val_metrics, test_metrics

if __name__ == "__main__":
    val_metrics, test_metrics = main()

### analyze_model_outputs

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, GemmaForCausalLM
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

# Configuration
MODEL_PATH = "/home/liorkob/M.Sc/thesis/Gemma7b/gemma-citation-ft/final_model"
TEST_CSV = "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv"

def analyze_model_outputs():
    """Analyze what the model is actually generating"""
    
    print("🔍 Loading model for detailed analysis...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    model = GemmaForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    model.eval()
    
    # Load test data
    df_test = pd.read_csv(TEST_CSV)
    
    print(f"📊 Dataset Analysis:")
    print(f"Total examples: {len(df_test)}")
    print(f"Label distribution:")
    print(df_test['label'].value_counts())
    print(f"Class balance: {df_test['label'].value_counts(normalize=True)}")
    
    # Test different prompts
    test_prompts = [
        "Question: Should these cases cite each other? Answer:",
        "Should these cases cite each other?",
        "Citation prediction:",
        "Answer yes or no:",
        "Predict:"
    ]
    
    print(f"\n🧪 Testing different prompts:")
    print("=" * 60)
    
    for prompt in test_prompts:
        print(f"\nPrompt: '{prompt}'")
        
        # Test with a few examples
        outputs = []
        for i in range(5):
            input_text = prompt
            
            # Tokenize
            input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
            
            # Generate
            with torch.no_grad():
                generated = model.generate(
                    input_ids,
                    max_new_tokens=20,
                    do_sample=False,
                    temperature=0.1,
                    pad_token_id=tokenizer.pad_token_id
                )
            
            # Decode
            response = tokenizer.decode(generated[0][len(input_ids[0]):], skip_special_tokens=True)
            outputs.append(response.strip())
        
        print(f"Outputs: {outputs}")
    
    return model, tokenizer

def test_with_better_generation_params(model, tokenizer):
    """Test with different generation parameters"""
    
    print(f"\n🎛️ Testing different generation parameters:")
    print("=" * 60)
    
    prompt = "Question: Should these cases cite each other? Answer:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    
    generation_configs = [
        {"max_new_tokens": 3, "do_sample": False, "temperature": 1.0},
        {"max_new_tokens": 5, "do_sample": True, "temperature": 0.3, "top_p": 0.8},
        {"max_new_tokens": 1, "do_sample": False},
        {"max_new_tokens": 10, "do_sample": False, "repetition_penalty": 1.2}
    ]
    
    for i, config in enumerate(generation_configs):
        print(f"\nConfig {i+1}: {config}")
        
        outputs = []
        for _ in range(3):
            with torch.no_grad():
                generated = model.generate(
                    input_ids,
                    pad_token_id=tokenizer.pad_token_id,
                    **config
                )
            response = tokenizer.decode(generated[0][len(input_ids[0]):], skip_special_tokens=True)
            outputs.append(response.strip())
        
        print(f"Outputs: {outputs}")

def analyze_training_data_format():
    """Analyze if there are issues with training data format"""
    
    print(f"\n📋 Analyzing training data format:")
    print("=" * 60)
    
    # Check all splits
    for split_name, csv_path in [
        ("Train", "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_train.csv"),
        ("Val", "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_val.csv"),
        ("Test", "/home/liorkob/M.Sc/thesis/citation-prediction/data_splits/crossencoder_test.csv")
    ]:
        try:
            df = pd.read_csv(csv_path)
            print(f"\n{split_name} set:")
            print(f"  Size: {len(df)}")
            print(f"  Columns: {list(df.columns)}")
            print(f"  Label distribution: {df['label'].value_counts().to_dict()}")
            print(f"  Label types: {df['label'].dtype}")
            print(f"  Sample labels: {df['label'].head(10).tolist()}")
            
            # Check for any text in label column
            unique_labels = df['label'].unique()
            print(f"  Unique labels: {unique_labels}")
            
        except Exception as e:
            print(f"  Error loading {split_name}: {e}")

def create_improved_evaluation_dataset():
    """Create a better evaluation approach"""
    
    class ImprovedEvalDataset(Dataset):
        def __init__(self, dataframe, tokenizer, max_len=512):
            self.data = dataframe.reset_index(drop=True)
            self.tokenizer = tokenizer
            self.max_len = max_len
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            row = self.data.iloc[idx]
            
            # Get true label
            label_str = str(row["label"]).strip().lower()
            if label_str in ["1", "yes", "1.0"]:
                true_label = 1
            else:
                true_label = 0
            
            # Simple, clear prompt
            prompt = "Answer yes or no: "
            
            # Tokenize
            prompt_tokens = self.tokenizer.encode(prompt, add_special_tokens=True)
            input_ids = prompt_tokens + [self.tokenizer.pad_token_id] * (self.max_len - len(prompt_tokens))
            input_ids = input_ids[:self.max_len]
            
            attention_mask = [1] * len(prompt_tokens) + [0] * (self.max_len - len(prompt_tokens))
            attention_mask = attention_mask[:self.max_len]
            
            return {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "true_label": true_label,
                "prompt_length": len(prompt_tokens),
                "original_text": row.get("text", ""),  # If available
                "row_index": idx
            }
    
    return ImprovedEvalDataset

def improved_prediction_function(model, tokenizer, input_ids, attention_mask, prompt_length):
    """Improved prediction with better parsing"""
    
    with torch.no_grad():
        # Try multiple generation approaches
        approaches = [
            {"max_new_tokens": 1, "do_sample": False},
            {"max_new_tokens": 3, "do_sample": False, "temperature": 0.1},
            {"max_new_tokens": 5, "do_sample": False}
        ]
        
        all_outputs = []
        
        for config in approaches:
            try:
                generated = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    **config
                )
                
                generated_text = tokenizer.decode(
                    generated[0][prompt_length:], 
                    skip_special_tokens=True
                ).strip().lower()
                
                all_outputs.append(generated_text)
                
            except Exception as e:
                all_outputs.append(f"error: {e}")
        
        # Parse outputs with multiple strategies
        for output in all_outputs:
            if output.startswith('yes') or output == 'y':
                return 1, 0.8, output
            elif output.startswith('no') or output == 'n':
                return 0, 0.8, output
        
        # If no clear yes/no, default to no
        return 0, 0.3, all_outputs[0] if all_outputs else "no_output"

def run_improved_evaluation():
    """Run evaluation with improved methods"""
    
    print(f"\n🚀 Running improved evaluation:")
    print("=" * 60)
    
    model, tokenizer = analyze_model_outputs()
    test_with_better_generation_params(model, tokenizer)
    analyze_training_data_format()
    
    # Load test data
    df_test = pd.read_csv(TEST_CSV)
    
    # Create improved dataset
    ImprovedEvalDataset = create_improved_evaluation_dataset()
    test_dataset = ImprovedEvalDataset(df_test, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    print(f"\n🧮 Running improved predictions on {len(df_test)} examples...")
    
    predictions = []
    true_labels = []
    generated_texts = []
    detailed_outputs = []
    
    model.eval()
    
    # Test on a subset first
    test_samples = min(50, len(df_test))
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_loader, desc="Evaluating")):
            if i >= test_samples:
                break
                
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            true_label = batch["true_label"].item()
            prompt_length = batch["prompt_length"].item()
            
            pred, conf, gen_text = improved_prediction_function(
                model, tokenizer, input_ids, attention_mask, prompt_length
            )
            
            predictions.append(pred)
            true_labels.append(true_label)
            generated_texts.append(gen_text)
            
            detailed_outputs.append({
                'true': true_label,
                'pred': pred,
                'confidence': conf,
                'generated': gen_text,
                'correct': pred == true_label
            })
    
    # Analysis
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    
    accuracy = (predictions == true_labels).mean()
    
    print(f"\n📈 Improved Results (on {test_samples} samples):")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Predictions distribution: {np.bincount(predictions)}")
    print(f"True labels distribution: {np.bincount(true_labels)}")
    
    print(f"\n🔍 Sample outputs:")
    for i in range(min(10, len(detailed_outputs))):
        out = detailed_outputs[i]
        status = "✅" if out['correct'] else "❌"
        print(f"{i+1}. Generated: '{out['generated']}' | Pred: {out['pred']} | True: {out['true']} {status}")
    
    # Identify patterns
    correct_mask = predictions == true_labels
    print(f"\n🎯 Pattern Analysis:")
    print(f"Correct predictions: {correct_mask.sum()}/{len(correct_mask)} ({correct_mask.mean():.2%})")
    
    # Group by prediction
    for pred_val in [0, 1]:
        mask = predictions == pred_val
        if mask.sum() > 0:
            accuracy_for_pred = (true_labels[mask] == pred_val).mean()
            print(f"When predicting {pred_val}: {mask.sum()} cases, {accuracy_for_pred:.2%} accuracy")
    
    return detailed_outputs

# Run the comprehensive analysis
if __name__ == "__main__":
    detailed_outputs = run_improved_evaluation()